In [47]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits import mplot3d
import capsol as cs
import scifi


In [48]:
# %pip install mayavi  

In [49]:
r = np.r_[np.arange(0, 50), np.arange(52.5, 150, 5)]

In [50]:
z = np.r_[np.arange(0, 50), np.arange(52.5, 200, 5)]

In [51]:

Rtip = 20.0
rcant = 500.0
hcone = 100.0
dcant = 10.0
theta = np.pi/180*15.0

ang = np.arange(10) * np.pi*2/10

R, Z, A = np.meshgrid(r, z, ang)
X = R * np.cos(A)
Y = R*np.sin(A)

In [52]:
sphere = cs.sphere(R, Z, Rtip)

In [53]:
def XYZ(X, Y, Z, tip):
    xt, yt, zt = X[tip], Y[tip], Z[tip]
    xyz = np.c_[xt.ravel(), yt.ravel(), zt.ravel()]
    xyz_u = np.unique(xyz, axis=0)
    return scifi.XYZFile(['H' for x in range(len(xyz_u))], xyz_u)

In [54]:
scifi.write_file('sphere.xyz', XYZ(X, Y, Z, sphere).write_str())

In [55]:
cone =  cs.cone(R, Z, Rtip, theta, hcone)
scifi.write_file('cone.xyz', XYZ(X, Y, Z, cone).write_str())

In [56]:
disk = cs.body(R, Z, hcone, dcant, rcant)
scifi.write_file('disk.xyz', XYZ(X, Y, Z, disk).write_str())

In [57]:
spm_tip = np.logical_or(np.logical_or(sphere, cone), disk)

In [58]:

scifi.write_file('spm_tip.xyz', XYZ(X, Y, Z, spm_tip).write_str())

# Normal grid

- Below, we try this for a normal (large) grid as used in the simulation!

In [59]:
nuni = 50 #number of uniformly spaced grid points
n = 500 # number of radial grid points
m = 500 # number of z+ grid points
h0 = 0.5 # initial grid spacing
rho_max = 1e6 # maximum radial size
z_max = 1e6 # maximum height size
z = 10.0 # tip-sample separation
l_js = 20  #m-, grid points in sample
l = l_js + int(np.round(z/h0)) # actual number of negative z points
verbose=0 
hsam = 15.0 #sample thickness
rtip = 20.0 #tip radius
theta = 15 * np.pi/180 #hallf angle of tip
hcone = 15000.0 # height of cone
rcant = 15000.0 # cantilever radius
dcant = 3000.0 #cantilever thickness
eps_r = 3.0 #dielec tric constant 

In [60]:
r,dr=cs.guni_grid(nuni, n, h0, rho_max)
z_plus, dz_plus= cs.guni_grid(nuni, m, h0, z_max)

In [61]:
z_minus=cs.generate_gapsam_grid(h0, hsam, z)
z_grid=np.r_[z_minus, z_plus]
# dz_minus=np.arange(N)*h0 
R, Z=np.meshgrid(r, z_grid)
# X = R * np.cos(A)
# Y = R*np.sin(A)

In [62]:
spm_tip = cs.cone(R, Z, Rtip, theta, hcone) + cs.sphere(R, Z, Rtip) + cs.body(R, Z, hcone, dcant, rcant)

In [63]:
first_z = np.ones_like(r, dtype=int)*-1
print(len(first_z))
for i, row in enumerate(spm_tip.T):
    if any(row):
        first_z[i] = np.argmax(row)

501


In [64]:
surf = []
for fz, r_ in zip(first_z, r):
    if fz >= 0:
        surf.append([r_, z_grid[fz]])

In [65]:
surf_ang = []
for row in surf:
    for angle in np.linspace(0, 2*np.pi, 12)[:-1]:
        surf_ang.append([row[0] * np.cos(angle), row[0] * np.sin(angle), row[1]])
surf_ang = np.array(surf_ang)

In [66]:
# scifi.write_file('spm_tip2.xyz', XYZ(X, Y, Z, spm_tip).write_str())

In [67]:
# R < np.sin(theta) * (Z-Rtip) + Rtip

In [68]:
# mm = (Rtip < Z) & (Z < hcone)
# mcone = R < np.sin(theta) * (Z-Rtip) + Rtip

In [69]:
# mmcone = mm * mcone

In [70]:
# scifi.write_file('spm_tip2.xyz', XYZ(X, Y, Z, mmcone).write_str())

In [74]:
%gui qt
from mayavi import mlab

In [ ]:
mlab.figure(1, fgcolor=(0, 0, 0), bgcolor=(1, 1, 1))

# Visualize the points
pts = mlab.points3d(*surf_ang.T, surf_ang.T[2])

# Create and visualize the mesh
mesh = mlab.pipeline.delaunay2d(pts)
surf_ = mlab.pipeline.surface(mesh)

# mlab.view(47, 57, 8.2, (0.1, 0.15, 0.14))
mlab.show()

qt.qpa.window: <QNSWindow: 0x7ff3bbebb6f0; contentView=<QNSView: 0x7ff3bbebbaa0; QCocoaWindow(0x7ff3bbec79c0, window=QWidgetWindow(0x7ff3bbed6630, name="QMainWindowClassWindow"))>> has active key-value observers (KVO)! These will stop working now that the window is recreated, and will result in exceptions when the observers are removed. Break in QCocoaWindow::recreateWindowIfNeeded to debug.


In [73]:
np.argmax((z_grid < 15500) * (z_grid > 15000))

376

In [46]:
z_grid[374:378]

array([14543.27093066, 14895.41556261, 15256.08913753, 15625.49822645])